## Step 1: Check GPU and Setup

In [ ]:
import torch
import os

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("WARNING: No GPU detected. Go to Runtime > Change runtime type and select GPU.")

# Set working directory
os.chdir('/content')
os.makedirs('DynamicBind', exist_ok=True)
os.chdir('/content/DynamicBind')
print(f"Working directory: {os.getcwd()}")

## Step 2: Install Dependencies

In [ ]:
print("Installing PyTorch...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("Installing PyTorch Geometric...")
!pip install -q torch-geometric

print("Installing RDKit and other dependencies...")
!pip install -q rdkit biopython pandas numpy scipy scikit-learn matplotlib tqdm pyyaml requests

print("Installing ESM and spyrmsd...")
!pip install -q fair-esm spyrmsd e3nn

print("\n✓ All dependencies installed successfully")

## Step 3: Clone/Setup DynamicBind Repository

In [ ]:
import subprocess
import os

os.chdir('/content')

# Clone from original repo (or you can upload a ZIP)
if not os.path.exists('DynamicBind'):
    !git clone https://github.com/luwei0917/DynamicBind.git
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

os.chdir('/content/DynamicBind')
print(f"Working directory: {os.getcwd()}")

## Step 4: Download Model Checkpoints from Zenodo

In [ ]:
import os
import subprocess

os.chdir('/content/DynamicBind')
os.makedirs('workdir', exist_ok=True)

# Download checkpoint from Zenodo (v2)
checkpoint_url = 'https://zenodo.org/records/10183369/files/workdir.zip'
checkpoint_file = 'workdir.zip'

if not os.path.exists(os.path.join('workdir', 'big_score_model_sanyueqi_with_time')):
    print(f"Downloading model checkpoint from Zenodo (may take 5-10 minutes)...")
    !wget -O {checkpoint_file} {checkpoint_url}
    print("Extracting...")
    !unzip -o {checkpoint_file} -d workdir
    # Fix nested directory structure if needed
    if os.path.exists('workdir/workdir'):
        !mv workdir/workdir/* workdir/
        !rmdir workdir/workdir
    print("✓ Checkpoints ready")
    !rm {checkpoint_file}
else:
    print("✓ Checkpoints already available")

print(f"Workdir contents: {os.listdir('workdir')}")

## Step 5: Upload Input Files (PDB & SDF)

In [ ]:
from google.colab import files
import os

os.makedirs('/content/DynamicBind/input', exist_ok=True)
os.chdir('/content/DynamicBind')

print("Upload your protein PDB file and ligand SDF file.")
print("Click 'Choose Files' and select both files.\n")

uploaded = files.upload()

pdb_file = None
sdf_file = None

for fname in uploaded.keys():
    if fname.endswith('.pdb'):
        pdb_file = f'input/{fname}'
        os.rename(fname, pdb_file)
    elif fname.endswith('.sdf'):
        sdf_file = f'input/{fname}'
        os.rename(fname, sdf_file)

if pdb_file and sdf_file:
    print(f"✓ PDB: {pdb_file}")
    print(f"✓ SDF: {sdf_file}")
else:
    print("ERROR: Please upload both PDB and SDF files.")
    if pdb_file:
        print(f"  PDB found: {pdb_file}")
    if sdf_file:
        print(f"  SDF found: {sdf_file}")

## Step 6: Prepare Input CSV

In [ ]:
import pandas as pd
import os

os.chdir('/content/DynamicBind')

# Create input CSV
pdb = os.path.abspath('input/unnamed.pdb') if os.path.exists('input/unnamed.pdb') else None
sdf = os.path.abspath('input/Conformer3D_COMPOUND_CID_977.sdf') if os.path.exists('input/Conformer3D_COMPOUND_CID_977.sdf') else None

# Auto-detect uploaded files
for f in os.listdir('input'):
    if f.endswith('.pdb'):
        pdb = os.path.abspath(f'input/{f}')
    elif f.endswith('.sdf'):
        sdf = os.path.abspath(f'input/{f}')

if pdb and sdf:
    df = pd.DataFrame({'ligand': [sdf], 'protein_path': [pdb]})
    df.to_csv('data/input_compounds.csv', index=False)
    print(f"✓ Input CSV created")
    print(df)
else:
    print(f"ERROR: Missing files. PDB={pdb}, SDF={sdf}")

## Step 7: Run DynamicBind Inference

In [ ]:
import subprocess
import os
import sys

os.chdir('/content/DynamicBind')

# Prepare Python command
python_bin = sys.executable  # Use Colab's Python

# Run inference with GPU (no relaxation to save time)
cmd = [
    python_bin,
    'run_single_protein_inference.py',
    'data/input_compounds.csv',
    'data/input_compounds.csv',
    '--protein_path_in_ligandFile',
    '--no_clean',
    '--ligand_is_sdf',
    '--no_relax',  # Skip relaxation to save time
    '--samples_per_complex', '10',  # Number of poses
    '--savings_per_complex', '10',
    '--inference_steps', '20',
    '--header', 'colab_run',
    '--python', python_bin,
    '--relax_python', python_bin,
    '--device', '0'
]

print(f"Running DynamicBind inference...\nCommand: {' '.join(cmd)}\n")
result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("\n✓ Inference complete!")
else:
    print(f"\n⚠ Inference finished with return code {result.returncode}")

## Step 8: Check Results

In [ ]:
import os
import subprocess

os.chdir('/content/DynamicBind')

# List results
results_dir = 'results/colab_run'
if os.path.exists(results_dir):
    print(f"Results directory: {results_dir}\n")
    result = subprocess.run(['find', results_dir, '-type', 'f'], capture_output=True, text=True)
    print(result.stdout)
    
    # Show summary
    print("\nGenerated files:")
    !du -sh results/colab_run/* 2>/dev/null | head -20
else:
    print(f"Results directory not found: {results_dir}")
    print("Available directories:")
    !ls -la results/ 2>/dev/null || echo "No results directory"

## Step 9: Download Results

In [ ]:
from google.colab import files
import shutil
import os

os.chdir('/content/DynamicBind')

results_dir = 'results/colab_run'
if os.path.exists(results_dir):
    # Create a ZIP of results
    print(f"Preparing results for download...")
    shutil.make_archive('DynamicBind_results', 'zip', results_dir)
    
    # Download
    print(f"Downloading results...")
    files.download('DynamicBind_results.zip')
    print("\n✓ Download complete! Check your Downloads folder.")
else:
    print(f"ERROR: Results directory not found. Check inference output above for errors.")

## Summary

You have completed DynamicBind inference on Google Colab!

**Output files include:**
- Ranked poses (`.sdf` and `.pdb` files) for the ligand-protein complex
- Affinity predictions (`.csv`)
- Intermediate data for visualization (`.pkl` files)

**Next steps:**
1. Download the ZIP file with all results
2. Extract and analyze the predicted complexes in your preferred molecular viewer (e.g., PyMOL, Chimera)
3. Evaluate binding affinities and pose quality

For more details, see: [DynamicBind Paper](https://www.nature.com/articles/s41467-024-45461-2)